# Nullspace BERT Demonstration

##### Install Requirements

In [1]:
!pip install -r requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.3/738.3 KB 9.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 14.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 14.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.8/925.8 KB 12.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 14.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 15.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

---

In [2]:
import numpy as np
from tqdm import tqdm
import transformers
from transformers import (
    BertTokenizer,
    BertForMaskedLM
)
import pandas as pd
import torch
from torch.nn import functional as F
from transformers import logging

logging.set_verbosity_error()

# BERT Transformer

In [3]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_mlm = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict=True)

Downloading: 100%|██████████| 232k/232k [00:00<00:00, 2.53MB/s]
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 10.0kB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 3.05MB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 242kB/s]
Downloading: 100%|██████████| 440M/440M [00:45<00:00, 9.68MB/s] 


![title](images/bert_architecture.png)

Source: [**BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding**: Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova](https://arxiv.org/abs/1810.04805)

In [7]:
BERT_BIAS_LAYER = 1

def chunker(input_list, chunk_size):
    """split sequence into chunks"""
    for i in range(0, len(input_list), chunk_size):
        yield input_list[i:i + chunk_size]
        
def get_embeddings(input_sequences, model, tokenizer):
    """extract hidden state from nth attention layer in encoder as specified by BERT_BIAS_LAYER"""
    tokenized_input = bert_tokenizer.batch_encode_plus(input_sequences, return_tensors = "pt", padding=True, truncation=False)
    embeddings = bert_mlm(**tokenized_input, output_hidden_states=True).hidden_states[BERT_BIAS_LAYER]
    return embeddings.detach().numpy(), tokenized_input["input_ids"].detach().numpy()

def extract_token_embeddings(embeddings, input_ids):
    """filter special token embeddings"""
    extracted_embeddings = []
    for idx in range(embeddings.shape[0]):
        if 0 in input_ids[idx]: # if input contains padding
            eos_idx = list(input_ids[idx]).index(0) - 1
        else:
            eos_idx = list(input_ids[idx]).index(102)
        extracted_embeddings.append(embeddings[idx][1:eos_idx].mean(axis=0))
    return np.array(extracted_embeddings)

# BERT Vector Generation

![title](images/bert_layers_gender_bias.png)

Source: [**Investigating Gender Bias in BERT**: Rishabh Bhardwaj, Navonil Majumder, Soujanya Poria](https://arxiv.org/abs/2009.05021)

In [8]:
VOCABULARY = pd.read_csv("data/vocabulary.txt", header=None)[0].values.tolist()

In [19]:
import gensim
from gensim import downloader
from gensim.models import KeyedVectors
# word2vec = gensim.downloader.load('glove-wiki-gigaword-300')
# word2vec.save('vectors.kv')
word2vec = KeyedVectors.load('vectors.kv')
print(type(word2vec.key_to_index))
print(len(word2vec.key_to_index))

<class 'dict'>
400000


In [9]:
bert_vocab_embedding_list = np.empty((0, 768))
for chunk in tqdm(chunker(VOCABULARY, 1000)):
    embeddings, input_ids = get_embeddings(chunk, bert_mlm, bert_tokenizer)
    embeddings = extract_token_embeddings(embeddings, input_ids)
    bert_vocab_embedding_list = np.concatenate((bert_vocab_embedding_list, embeddings), axis=0)


8it [02:41, 20.19s/it]


In [10]:
embedding_shape = bert_vocab_embedding_list.shape
with open("data/embeddings/BERTLM_ENCODER_LAYER_ONE/bert-base-uncased-embeddings.txt", "w") as bert_file:
    bert_file.write(f"{embedding_shape[0]} {embedding_shape[1]} \n")
    for word, embedding in zip(VOCABULARY, bert_vocab_embedding_list):
        bert_file.write(f"{word} {' '.join(map(str, list(embedding)))}\n")


In [11]:
!python get_bias_sensitive_tokens.py

pca explained variance ratio: [0.32702196 0.20407493 0.15900119 0.0817484  0.05758673 0.05069628
 0.04543994 0.03472264 0.0293372  0.01037071] 

TOP 100 MALE SENSITIVE TOKENS 
 ('man', 'john', 'he', 'boy', 'guy', 'son', 'his', 'manhunt', 'him', 'housman', 'himself', 'heisman', 'manu', 'johny', 'heyman', 'sons', 'hes', 'guymon', 'sonnen', 'boye', 'brothers', 'rockman', 'mangini', 'father', 'brother', 'mike', 'sandman', 'bossman', 'heder', 'jono', 'helt', 'hedi', 'hegel', 'dude', 'bluesman', 'jason', 'mr', 'sonali', 'redman', 'sonographer', 'linesman', 'charles', 'cashman', 'handsome', 'boyhood', 'ferdinando', 'marksman', 'darkman', 'timo', 'kingdon', 'cocky', 'walter', 'sonja', 'james', 'robert', 'heba', 'matthew', 'irishman', 'pitman', 'martino', 'paulus', 'mikel', 'juwan', 'tradesman', 'ockham', 'welshman', 'bagman', 'edgar', 'hegelian', 'vorderman', 'richard', 'william', 'david', 'countryman', 'bouchon', 'kings', 'waltrip', 'rotman', 'samo', 'heliotrope', 'wilfredo', 'cheeseman', 'th

# Null-Space Projection

In [12]:
!python context_nullspace_projection.py

Traceback (most recent call last):
  File "/Users/cjy/Desktop/Gender-Debiasing-in-BERT-mybranch/context_nullspace_projection.py", line 6, in <module>
    import debias
ModuleNotFoundError: No module named 'debias'


![title](images/tsne_projections.png)

# Transformer Encoder / Decoder Generation

In [13]:
NULL_PROJECTION = np.load("data/nullspace_vector.npy")

FileNotFoundError: [Errno 2] No such file or directory: 'data/nullspace_vector.npy'

In [10]:
def guard_vector(layer):
    """Apply nullprojection to inputted vector"""
    return NULL_PROJECTION.dot(layer.T).T


def guard_embedding(hidden_state, tokenized_input):
    """Apply the linear guarding function to hidden state"""
    input_ids_numpy = list(tokenized_input["input_ids"].detach().numpy()[0])
    word_indexes = [input_ids_numpy.index(token_id) for token_id in input_ids_numpy if token_id not in [101, 103, 102, 0]]
    bias_layer_numpy =  hidden_state.detach().numpy()
    for idx in word_indexes:
        bias_layer_numpy[0][idx] = guard_vector(bias_layer_numpy[0][idx])
    return torch.Tensor(bias_layer_numpy)


def run_post_bias_encoder_layers(encoder_layers_list, previous_hidden_state):
    """Manually run embeddings through attention blocks in encoder"""
    for attention_block in encoder_layers_list:
        previous_hidden_state = attention_block.forward(hidden_states=previous_hidden_state)[0]
    return previous_hidden_state


def get_next_word(logits, tokenizer, mask_index):
    """Generate the next highest liklihood word given logits"""
    softmax = F.softmax(logits, dim = -1)
    mask_word = softmax[0, mask_index, :]
    top_word = torch.argmax(mask_word, dim=1)
    return tokenizer.decode(top_word)


def generate_next_word(input_sequence, model, tokenizer, guard_flag=False, biased_layer_index=1):
    # extracting modules from BERT LM
    bert_encoder_modules = list(bert_mlm.modules())[8:-5] # extract list of model components
    encoder_layers_list = [bert_encoder_modules[idx] for idx in range(19, 206, 17)] # extracting each encoder attention block
    bert_mlm_head = bert_encoder_modules[-1] # extracting BERT LM Head

    # tokenize input sequence
    tokenized_input = tokenizer.encode_plus(input_sequence, return_tensors = "pt")
    mask_index = torch.where(tokenized_input["input_ids"][0] == bert_tokenizer.mask_token_id)

    # extracting encoding and feeding back into model
    hidden_state = model(**tokenized_input, output_hidden_states=True).hidden_states[biased_layer_index]
    
    # apply guarding function to hidden state
    hidden_state = guard_embedding(hidden_state, tokenized_input) if guard_flag else hidden_state
    
    # run guarded hidden state through remaining encoder layers
    encoder_output = run_post_bias_encoder_layers(encoder_layers_list, hidden_state)
    
    # pass encoder output into LM Head to generate logits
    output_logits = bert_mlm_head.forward(sequence_output=encoder_output)

    # generate the highest likelihood word
    return get_next_word(output_logits, tokenizer, mask_index)


def complete_the_sentence(female_variant, male_variant):
    guarded_pred_female = generate_next_word(female_variant, bert_mlm, bert_tokenizer, guard_flag=True)
    unguarded_pred_female = generate_next_word(female_variant, bert_mlm, bert_tokenizer, guard_flag=False)
    guarded_pred_male = generate_next_word(male_variant, bert_mlm, bert_tokenizer, guard_flag=True)
    unguarded_pred_male = generate_next_word(male_variant, bert_mlm, bert_tokenizer, guard_flag=False)

    guarded_female_sentence = female_variant.replace("[MASK]", guarded_pred_female.upper())
    unguarded_female_sentence = female_variant.replace("[MASK]", unguarded_pred_female.upper())
    guarded_male_sentence = male_variant.replace("[MASK]", guarded_pred_male.upper())
    unguarded_male_sentence = male_variant.replace("[MASK]", unguarded_pred_male.upper())
    output_string = (f"unguarded outputs: \n \t{unguarded_female_sentence} \n \t{unguarded_male_sentence} \n\n" +
                     f"guarded outputs: \n \t{guarded_female_sentence} \n \t{guarded_male_sentence} \n")
    print(output_string)



In [13]:
print("\n-------------- people thought <she/he> was very [MASK] during the meeting. -----------------\n")
complete_the_sentence("people thought she was very " + bert_tokenizer.mask_token + " during the meeting.",
                      "people thought he was very " + bert_tokenizer.mask_token + " during the meeting.")


-------------- people thought <she/he> was very [MASK] during the meeting. -----------------

unguarded outputs: 
 	people thought she was very BEAUTIFUL during the meeting. 
 	people thought he was very POPULAR during the meeting. 

guarded outputs: 
 	people thought she was very QUIET during the meeting. 
 	people thought he was very QUIET during the meeting. 

